In [10]:
# ==============================================================================
# Notebook 3 (v2): Otimização, SHAP e Interpretação
# ==============================================================================

# ------------------------------------------------------------------------------
# 1. Configuração e Carregamento de Bibliotecas e Dados
# ------------------------------------------------------------------------------
import pandas as pd
import numpy as np
import joblib
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json
import shap # <-- Importação da biblioteca SHAP

from sklearn.metrics import classification_report, roc_curve, roc_auc_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV # <-- Importação para otimização
from sklearn.ensemble import RandomForestClassifier # <-- Importação do modelo 

In [11]:
# --- Definindo os caminhos conforme a estrutura de diretórios ---
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

PROCESSED_SPLIT_PATH = os.path.join(ROOT_DIR, 'data', 'processed', 'split')
FINAL_MODEL_PATH = os.path.join(ROOT_DIR, 'models', 'final_model_v2') # <-- Novo diretório para a v2
REPORTS_PATH = os.path.join(ROOT_DIR, 'reports', 'v2') # <-- Novo diretório para relatórios da v2
PLOTS_PATH = os.path.join(ROOT_DIR, 'reports', 'v2', 'plots') # <-- Novo diretório para gráficos da v2

In [12]:
# Criando os diretórios se não existirem
os.makedirs(FINAL_MODEL_PATH, exist_ok=True)
os.makedirs(REPORTS_PATH, exist_ok=True)
os.makedirs(PLOTS_PATH, exist_ok=True)

In [13]:
# Carregando os conjuntos de treino e teste salvos no Notebook 2
print("Carregando conjuntos de treino e teste salvos...")
try:
    X_train = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'X_train.csv'))
    X_test = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'X_test.csv'))
    y_train = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'y_train.csv')).squeeze()
    y_test = pd.read_csv(os.path.join(PROCESSED_SPLIT_PATH, 'y_test.csv')).squeeze()
    print("✅ Dados de treino e teste carregados com sucesso.")
except FileNotFoundError:
    print(f"❌ Erro: Arquivos de split não encontrados. Certifique-se de ter executado o Notebook 2.")
    X_train, X_test, y_train, y_test = None, None, None, None

Carregando conjuntos de treino e teste salvos...
✅ Dados de treino e teste carregados com sucesso.


In [14]:
if X_train is not None:
    # ------------------------------------------------------------------------------
    # 2. Tratamento de Classes Desbalanceadas (SMOTE)
    # ------------------------------------------------------------------------------
    print("\nTratando o desbalanceamento de classes com SMOTE...")
    smote = SMOTE(random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    print("✅ SMOTE aplicado. Dimensão do conjunto de treino após SMOTE:")
    print(f"X_train_res: {X_train_res.shape} | y_train_res: {y_train_res.shape}")


Tratando o desbalanceamento de classes com SMOTE...
✅ SMOTE aplicado. Dimensão do conjunto de treino após SMOTE:
X_train_res: (7228, 30) | y_train_res: (7228,)


In [15]:
# ------------------------------------------------------------------------------
# 3. Otimização de Hiperparâmetros (Randomized Search)
# ------------------------------------------------------------------------------
print("\nIniciando a otimização de hiperparâmetros com Randomized Search...")
    
# Modelo base para a otimização
model_base = RandomForestClassifier(random_state=42)

# Parâmetros para Randomized Search
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

random_search = RandomizedSearchCV(
    model_base,
    param_distributions=param_dist,
    n_iter=10, 
    cv=3, 
    scoring='f1',
    random_state=42,
    n_jobs=-1,
    verbose=1
)
    
random_search.fit(X_train_res, y_train_res)
best_model_tuned = random_search.best_estimator_

print("\n✅ Otimização concluída.")
print(f"Melhores hiperparâmetros encontrados: {random_search.best_params_}")


Iniciando a otimização de hiperparâmetros com Randomized Search...
Fitting 3 folds for each of 10 candidates, totalling 30 fits

✅ Otimização concluída.
Melhores hiperparâmetros encontrados: {'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None}


In [16]:
# ------------------------------------------------------------------------------
# 4. Avaliação e Interpretação do Modelo Otimizado
# ------------------------------------------------------------------------------
print("\n--- Avaliação Final do Modelo Otimizado ---")
    
y_pred_proba = best_model_tuned.predict_proba(X_test)[:, 1]
    
# Otimização do limiar
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
f1_scores = [f1_score(y_test, y_pred_proba > t, zero_division=0) for t in thresholds]
best_threshold = thresholds[np.argmax(f1_scores)]
y_pred_optimized = (y_pred_proba >= best_threshold).astype(int)
    
# Gerando métricas de forma estruturada para JSON
report_dict = classification_report(y_test, y_pred_optimized, output_dict=True)
final_metrics = {
    'accuracy': report_dict['accuracy'],
    'precision': report_dict['macro avg']['precision'],
    'recall': report_dict['macro avg']['recall'],
    'f1-score': report_dict['macro avg']['f1-score'],
    'auc_score': roc_auc_score(y_test, y_pred_proba),
    'best_threshold': best_threshold,
    'best_params': random_search.best_params_
}
    
with open(os.path.join(REPORTS_PATH, 'final_model_metrics_v2.json'), 'w') as f:
    json.dump(final_metrics, f, indent=4)
print("✅ Métricas do modelo final v2 salvas em 'final_model_metrics_v2.json'.")


--- Avaliação Final do Modelo Otimizado ---
✅ Métricas do modelo final v2 salvas em 'final_model_metrics_v2.json'.


In [18]:
# ------------------------------------------------------------------------------
# 5. Interpretação do Modelo com SHAP
# ------------------------------------------------------------------------------
print("\n--- Interpretação do Modelo com SHAP ---")

# Criando o explainer SHAP
explainer = shap.TreeExplainer(best_model_tuned)
shap_values = explainer.shap_values(X_test)
    
# Salvando o explainer para uso futuro
joblib.dump(explainer, os.path.join(FINAL_MODEL_PATH, 'shap_explainer_v2.pkl'))
print("✅ Explainer SHAP salvo com sucesso.")

# Usando shap_values[1] para a classe de interesse
shap.summary_plot(shap_values[1], features=X_test, show=False)
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_PATH, 'shap_summary_plot_v2.png'))
plt.close() # <-- Fechar o plot para não exibir no notebook
print("✅ Gráfico de resumo SHAP salvo em 'shap_summary_plot_v2.png'.")
    
# Salvando o modelo final otimizado
joblib.dump(best_model_tuned, os.path.join(FINAL_MODEL_PATH, 'random_forest_final_v2.pkl'))
print("✅ Modelo final otimizado v2 salvo com sucesso.")
    
print("\nProcesso do Notebook 3 (v2) concluído com sucesso!")


--- Interpretação do Modelo com SHAP ---
✅ Explainer SHAP salvo com sucesso.


AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.